# Defining optimization problems 
In this notebook, we show how optimization problems can be defined in MetaJul. We focus first on continuous problems and after that we include examples for binary problems.

## Continuous problems
A continuous problem is characterized by:

* A vector of ``Bounds`` objects, specifying the lower and upper bounds of the decision variables, which can be integer or real values
* A vector containing the objective values
* A vector contining the overall constraint violation values
* The problem name

so the point is how to create a problem including these elements. We use two approaches, which are ilustranted next. Both approaches make use of a ``createSolution()`` function that creates a solution whose variables are randomly initialized from the problem bounds:

```julia
function createSolution(problem::AbstractContinuousProblem{T})::ContinuousSolution{T} where {T<:Number}
  x = [problem.bounds[i].lowerBound + rand() * (problem.bounds[i].upperBound - problem.bounds[i].lowerBound) for i in 1:length(problem.bounds)]

  return ContinuousSolution{T}(x, zeros(numberOfObjectives(problem)), zeros(numberOfConstraints(problem)), Dict(), problem.bounds)
end
```

### Method 1: Using the struct ``ContinuousProblem``

The first approach is to create them in a dynamic way by creating an instance of the ``ContinuousProblem`` struct defined in ``src/problem/continuousProblem.jl``:
```julia
abstract type AbstractContinuousProblem{T<:Number} <: Problem{T} end

mutable struct ContinuousProblem{T} <: AbstractContinuousProblem{T}
  bounds::Vector{Bounds{T}}
  objectives::Vector{Function}
  constraints::Vector{Function}
  name::String
end
```

Let us suppose that we intend to implement problem Schaffer, a continuous unconstrained float problem:

\begin{align}
f_1(x) & = x \\
f_2(x) & = (x-2)^2 \\
x & \in [-10^3, 10^3] 
\end{align}

We can create the problem in this way:

In [8]:
using MetaJul

schaffer = ContinuousProblem{Real}("Schaffer")

f = x -> x[1] * x[1]
g = x -> (x[1] - 2.0) * (x[1] - 2.0)

addObjective(schaffer, f) 
addObjective(schaffer, g)

addVariable(schaffer, Bounds{Real}(-1000.0, 1000.0)) ;

As this is a benchmark problem, we can encapsulate this code into a function to avoid having to define it whenever we intend to solve it: 

```julia
function schaffer()
  problem = ContinuousProblem{Real}("Schaffer")

  f = x -> x[1] * x[1]
  g = x -> (x[1] - 2.0) * (x[1] - 2.0)

  addObjective(problem, f)
  addObjective(problem, g)
    
  addVariable(problem, Bounds{Real}(-1000.0, 1000.0))

  return problem
end

```

To get the elements of a problem created in this way, the following functions are provided:
```julia

function numberOfVariables(problem::ContinuousProblem{T}) where {T}
  return length(problem.bounds)
end

function numberOfObjectives(problem::ContinuousProblem{T}) where {T}
  return length(problem.objectives)
end

function numberOfConstraints(problem::ContinuousProblem{T}) where {T}
  return length(problem.constraints)
end

function name(problem::ContinuousProblem{T}) where {T}
  return problem.name
end

function bounds(problem::ContinuousProblem{T}) where {T}
  return problem.bounds
end

```

In [9]:
println("Number of variables: ", numberOfVariables(schaffer))
println("Number of objectives: ", numberOfObjectives(schaffer))
println("Number of constraints: ", numberOfConstraints(schaffer))
println("Name: ", name(schaffer))
println("Bounds: ", bounds(schaffer));

Number of variables: 1
Number of objectives: 2
Number of constraints: 0
Name: Schaffer
Bounds: Bounds{Real}[Bounds{Real}(-1000.0, 1000.0)]


When a new solution is created, it must be evaluated to compute its objective values from the problem. The function for solution evaluations is common to all the problems created using the ``ContinuousProblem``struct: 
```julia
function evaluate(solution::ContinuousSolution{T}, problem::ContinuousProblem{T})::ContinuousSolution{T} where {T<:Number}
  for i in 1:length(problem.objectives)
    solution.objectives[i] = problem.objectives[i](solution.variables)
  end

  for i in 1:length(problem.constraints)
    solution.constraints[i] = problem.constraints[i](solution.variables)
  end

  return solution
end

```

We use the same approach to implement the following integer constrained continuous problem:

\begin{align}
max f_1(\vec{x}) & = x_1 + x_2 \\
min f_2(\vec{x}) & = x_1 + 3x_2 \\
s.t. \\
2x_1 + 3x_2 & \leq 30 \\
3x_1 + 2x_2 & \leq 30 \\
x_1 - x_2 & \leq 5.5 \\
x & \in [0, 20]
\end{align}

To implement this problem in MetaJul we have take into account:

- All the functions are supposed to be minimized, so the first objective has to multiplied by -1
- The constraints must be in the form `expression >= 0`

The code is:

In [14]:
problem = ContinuousProblem{Int64}("integerProblem")

addVariable(problem, Bounds{Int64}(0, 20))  
addVariable(problem, Bounds{Int64}(0, 20)) 

f1 = x -> -1.0 * (x[1] + x[2]) # objective to maximize
f2 = x -> x[1] + 3 * x[2]      # objective to minimize

addObjective(problem, f1)
addObjective(problem, f2)

c1 = x -> -2 * x[1] - 3 * x[2] + 30.0
c2 = x -> -3 * x[1] - 2 * x[2] + 30.0
c3 = x -> -x[1] + x[2] + 5.5

addConstraint(problem, c1)
addConstraint(problem, c2)
addConstraint(problem, c3) ;

In [13]:
println("Number of variables: ", numberOfVariables(problem))
println("Number of objectives: ", numberOfObjectives(problem))
println("Number of constraints: ", numberOfConstraints(problem))
println("Name: ", name(problem))
println("Bounds: ", bounds(problem));

Number of variables: 2
Number of objectives: 2
Number of constraints: 3
Name: integerProblem
Bounds: Bounds{Int64}[Bounds{Int64}(0, 20), Bounds{Int64}(0, 20)]


### Method 2: Creating a specific struct for the problem

There are problems whose functions are not independent (e.g., the formulation of a function includes other function), so the approach of using the ``ContinuousProblem`` struct may not be apropriate. An example is the ZDT6 problem, in which the second objective contains the value of evaluation the first one:

$$
f_1(\vec{x}) = 1 - exp(-4x_1)sin^6(6{\pi}x_1) 
$$
$$
f_2(\vec{x}) = g(\vec{x})[1 - (f_1(\vec{x})/g(\vec{x}))^2]
$$
$$
g(\vec{x}) = 1 + 9[(\sum_{i=2}^{n}x_i)/(n - 1)]^0.25
$$
$$
x \in [0.0, 1.0]
$$

This problem can be implemented as follows. First, we create a struct called ``ZDT6`` to contain the problem bounds:
    
```julia
struct ProblemZDT6 <: AbstractContinuousProblem{Float64}
  bounds::Vector{Bounds{Float64}}
end
```

Next we define a function called ``ZDT6()`` which initializes the problem bounds and the set of functions to retrieve the problem data:

```julia
function ZDT6(numberOfVariables::Int=10)
  bounds = [Bounds{Float64}(0.0, 1.0) for _ in range(1, numberOfVariables)]

  return ProblemZDT6(bounds)
end

function numberOfVariables(problem::ProblemZDT6)
  return length(problem.bounds)
end

function numberOfObjectives(problem::ProblemZDT6)
  return 2
end

function numberOfConstraints(problem::ProblemZDT6)
  return 0
end

function bounds(problem::ProblemZDT6)
  return problem.bounds
end

function name(problem::ProblemZDT6)
  return "ZDT6"
end

```

The last step is to implement the ``evaluate`` method containing the code of the objective functions:
```julia
function evaluate(solution::ContinuousSolution{Float64}, problem::ProblemZDT6)::ContinuousSolution{Float64}
  x = solution.variables
  @assert length(x) == numberOfVariables(problem) "The number of variables of the solution to be evaluated is not correct"

  function evalG(x::Vector{Float64})
    g = sum(x[i] for i in range(2,length(x)))
    g = g / (length(x) - 1.0)

    g = ^(g, 0.25)
    g = 9.0 * g
    g = 1.0 + g

    return g
  end

  function evalH(v::Float64, g::Float64)
    return 1.0 - ^(v/g, 2.0)
  end

  f1 = 1.0 - exp(-4.0*x[1]) * ^(sin(6*pi*x[1]),6.0)
  g = evalG(x)
  h = evalH(f1, g)
  f2 = g * h
 
  solution.objectives = [f1, f2]

  return solution
end
```

## Binary problems
The approaches to define binary problems are similar to the aforementioned ones for continuous problems. We show two examples that use the first method.

The classical OneMax problem (maximizing the number of ones in a binary string) can be implemented in this way:

In [4]:
using MetaJul

numberOfBits = 512
problem = BinaryProblem(numberOfBits, "OneMax")

f = x -> -1.0 * length([i for i in x.bits if i])

addObjective(problem, f) ;

A random solution for the problem can be obtained by calling the `createSolution()` function:

In [8]:
binarySolution = createSolution(problem)
evaluate(binarySolution, problem)

println(binarySolution.variables) 
println(-1.0 * binarySolution.objectives) ;

MetaJul.BitVector(Bool[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,

A bi-objective variant intented to optimize the number of ones and the number of zeroes can be implemented in similar way (we encapsulate the problem definition in a function):

In [9]:
function oneZeroMax(numberOfBits::Int) 
  problem = BinaryProblem(numberOfBits, "OneZeroMax")

  f = x -> length([i for i in x.bits if i])
  g = y -> length([j for j in y.bits if !j])

  addObjective(problem, f)
  addObjective(problem, g)

  return problem
end


oneZeroMax (generic function with 1 method)